In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject//DataAndExperiments/Experiments/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue --n_splits 1 --split 0  --batch_size 5               

In [ ]:
def check_history(model_path, num_folds):
    from visualize.data_utils import get_data_generic
    return os.path.exists(os.path.join(model_path, "status.txt"))
    
def check_results(model_path, MS_list, num_folds):
    import os
    import pathlib
    import numpy as np
    currentDirectory = pathlib.Path(model_path)
    currentPattern = "fold-*"
    flag=True
    for fold_dir in currentDirectory.glob(currentPattern):
        fold = int(str(fold_dir).split("-")[-1])

        selection_metrics = ["best_loss", "best_balanced_accuracy", "last_checkpoint"]
        cnn_classification_dir = os.path.join(model_path, 'fold-%i' % fold, 'cnn_classification')
        
        for selection_metric in selection_metrics:
            modes = ['train', 'validation']
            for ms_el in MS_list:
                modes.append('test_' + ms_el)
                
            for mode in modes:
                if not os.path.exists(os.path.join(cnn_classification_dir, selection_metric,
                                                       '%s_image_level_metrics.tsv' % (mode))):
                    flag=False
                
    return flag
    
def check_complete_test(model_path, num_folds, MS_list):
    import json
    path_params = os.path.join(model_path, "commandline_train.json")
    return (check_history(model_path, num_folds) and check_results(model_path, MS_list, num_folds))
    


In [ ]:
def check_baesian_stat(model_path, MS_list, num_folds):
    import os
    import pathlib
    import numpy as np
    currentDirectory = pathlib.Path(model_path)
    currentPattern = "fold-*"
    flag=True
    for fold_dir in currentDirectory.glob(currentPattern):
        fold = int(str(fold_dir).split("-")[-1])

        selection_metrics = ["best_loss", "best_balanced_accuracy", "last_checkpoint"]
        cnn_classification_dir = os.path.join(model_path, 'fold-%i' % fold, 'cnn_classification')
        
        for selection_metric in selection_metrics:
            modes = ['test_' + ms_el for ms_el in MS_list]
                
            for mode in modes:
                if not os.path.exists(os.path.join(cnn_classification_dir, selection_metric,
                                                       '%s_image_level_stats.tsv' % (mode))):
                    flag=False
                
    return flag

In [ ]:
import matplotlib.pyplot as plt

def get_rows_and_cols(data):
    rows_matrix = {}
    cols_matrix = {}
    for data_type in data.keys():
        cols_matrix[data_type] = [selection_metric.replace("_", " ") for selection_metric in data[data_type].keys()]
        if data_type == "history":
            cols_matrix[data_type] = ["loss", "balanced_accuracy"]
        else:
            cols_matrix[data_type] = [selection_metric.replace("_", " ") for selection_metric in data[data_type].keys()]

        if data_type == "uncertainty_distribution":
            rows_matrix[data_type] = [test_MS.replace("_", " ") for test_MS in
                                      list(data[data_type][list(data[data_type].keys())[0]].groupby("mode", as_index=False, sort=False).groups.keys())]
        else:
            rows_matrix[data_type] = [None]

    num_rows = sum([len(rows_matrix[row]) for row in rows_matrix.keys()])
    num_cols = max([len(cols_matrix[col]) for col in cols_matrix.keys()])
    return rows_matrix, cols_matrix, num_rows, num_cols


def plot_history(args, data, fig, row, figshape):
    from .plot_utils import plot_history_ax

    for col, history_mode in enumerate(args.history_modes):
        ax = plt.subplot2grid(shape=figshape, loc=(row, col), fig=fig)
        plot_history_ax(ax, data, mode=history_mode, aggregation_type=args.aggregation_type)



def plot_results(args, data, fig, row, figshape):
    from .plot_utils import plot_results_ax, plot_results_agg_ax

    for col, selection_mode in enumerate(list(data.keys())):
        ax = plt.subplot2grid(shape=figshape, loc=(row, col), fig=fig)
        if args.aggregation_type is not "all":
            plot_results_ax(ax, data[selection_mode], args.result_metrics)
        else:
            plot_results_agg_ax(ax, data[selection_mode], args.result_metrics)

        ax.set_title(selection_mode)




def plot_uncertainty_distribution(args, data, fig, row, figshape):
    from .plot_utils import plot_catplot_ax, set_ylims_axes
    axes = []

    for col, selection_mode in enumerate(list(data.keys())):
        for j, (mode, mode_group) in enumerate(data[selection_mode].groupby("mode", as_index=False, sort=False)):
            ax = plt.subplot2grid(shape=figshape, loc=(row+j, col), fig=fig)
            plot_catplot_ax(ax,mode_group, args.uncertainty_metric, args.ba_inference_mode, args.catplot_type )
            ax.set_title(selection_mode+"; "+mode)
            axes.append(ax)

    set_ylims_axes(axes)


def plot_combined_plots(args, model_params, saved_file_path, data=None):
    import matplotlib.pyplot as plt

    readable_params = ['model', 'data_augmentation', 'batch_size', 'learning_rate', "loss", 'training MS']

    rows_matrix, cols_matrix, num_rows, num_cols = get_rows_and_cols(data)
    fig = plt.figure(figsize=((int(8 * num_cols), int(6 * num_rows))))

    row = 0
    for data_key in sorted(list(data.keys()), reverse=True):
        eval("plot_%s" % (data_key))(args, data=data[data_key], fig=fig, figshape=(num_rows, num_cols), row=row)
        row+=len(rows_matrix[data_key])

    str_suptitle = "\n Params: "
    for i, line in enumerate(readable_params):
        str_suptitle += line + ': ' + str(model_params[line]) + "; "
    str_suptitle +="\n"

    plt.suptitle(str_suptitle)

    # plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, wspace=0.1, hspace=0.1)
    plt.subplots_adjust( left=0.05, right=0.95, top=0.95, bottom=0.05,hspace=0.3)

    if saved_file_path is not None:
        plt.savefig(saved_file_path)
    else:
        plt.show()

    plt.close()


def plot_generic(
        args,
        training_MS,
):
    import pathlib
    import os
    import json
    import pandas as pd
    from .data_utils import get_data_generic

    currentDirectory = pathlib.Path(args.model_path)
    path_params = os.path.join(currentDirectory, "commandline_train.json")

    with open(path_params, "r") as f:
        params = json.load(f)

    params['training MS'] = training_MS
    args.bayesian=params["bayesian"]
    model_name = os.path.basename(os.path.normpath(currentDirectory))

    folder_name = ''
    for data_type in sorted(args.data_types):
        if data_type=="uncertainty_distribution":
            folder_name += '%s_uncertainty_%s' % (args.uncertainty_metric, args.catplot_type)
        else:
            folder_name += data_type
        folder_name += "_"
        
    data=[]
    for f in args.models: 
        data = get_data_generic(args)

    for fold_key in data.keys():

        if args.aggregation_type=="separate":
            folder_fold_name = os.path.join("separate_folds", "fold-%s"%fold_key)
        else:
            folder_fold_name = fold_key

        if args.output_path:
            saved_file_path = os.path.join(args.output_path, folder_fold_name, params["model"],  folder_name)
            os.makedirs(saved_file_path, exist_ok=True)
            saved_file_path=os.path.join(saved_file_path, model_name + '.png')
        else:
            saved_file_path=None

        plot_combined_plots(args, model_params=params, data=data[fold_key], saved_file_path=saved_file_path)


In [ ]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ['1.5T', "3T"]

MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'
num_folds_arr=[5]
isBayesian_arr=[True]
num_folds=5
merged_file=os.path.join(home_folder,"DataAndExperiments/Data/DataStat", "merge.tsv")

for isBayesian in isBayesian_arr:
    for MS in MS_main_list[:]:
        print("MS %s \n ____________________________________________________________________________________________"%MS)
        model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3", "ResNet50", "SEResNet50",]
        MS_list = MS_list_dict[MS]
        inference_modes=["mode", "mean"]
        results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_%d-fold"%num_folds, "Experiments_Bayesian" if isBayesian else "Experiments", 'Experiments-' + MS)
        model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_%d-fold/Experiments-%s"%(num_folds, MS), "NNs_Bayesian" if isBayesian else "NNs")

        for network in model_types[:]:
            model_dir = os.path.join(model_dir_general, network)
            # output_dir = pathlib.Path(output_dir)
            modelPatter = "subject_model*"
            folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

            for f in folders[:]:
                if check_complete_test(f, num_folds=num_folds, MS_list=MS_list):
#                     pass
                    print(f)
                    for inference_mode in inference_modes:
                        results_dir=os.path.join(results_folder_general, "%s_inference"%inference_mode)
                        if not check_baesian_stat(f, num_folds=num_folds, MS_list=MS_list):
                            !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py bayesian $f stat
                        !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results history --merged_file $merged_file --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results history --aggregation_type "separate" --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"

                else: 
                    pass
#                     print(f)

In [ ]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ["1.5T-3T", '1.5T', "3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'
num_folds_arr=[5]
isBayesian_arr=[False]
num_folds=5
merged_file=os.path.join(home_folder,"DataAndExperiments/Data/DataStat", "merge.tsv")
for isBayesian in isBayesian_arr:
    for MS in MS_main_list[:]:
        print("MS %s \n ____________________________________________________________________________________________"%MS)
        model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3", "ResNet50", "SEResNet50" ]
        MS_list = MS_list_dict[MS]
        inference_modes=["mode", "mean"]
        results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_%d-fold"%num_folds, "Experiments_Bayesian" if isBayesian else "Experiments", 'Experiments-' + MS)
        model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_%d-fold/Experiments-%s"%(num_folds, MS), "NNs_Bayesian" if isBayesian else "NNs")

        for network in model_types[:]:
            model_dir = os.path.join(model_dir_general, network)
            # output_dir = pathlib.Path(output_dir)
            modelPatter = "subject_model*"
            folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

            for f in folders[:]:
                if check_complete_test(f, num_folds=num_folds,MS_list=MS_list):
#                     pass
                    print(f)
                    
                    results_dir=results_folder_general
                    !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir results history  --merged_file $merged_file --aggregation_type "all"

                else: 
                    pass
#                     print(f)

In [ ]:
def translate_parameters(args):
    """
    Translate the names of the parameters between command line and source code.
    """
    args.gpu = False
    args.num_workers = args.nproc
    args.optimizer = "Adam"
    args.batch_size=9
    # args.loss = "default"

    if hasattr(args, "caps_dir"):
        args.input_dir = args.caps_dir
    if hasattr(args, "unnormalize"):
        args.minmaxnormalization = not args.unnormalize
    if hasattr(args, "slice_direction"):
        args.mri_plane = args.slice_direction
    if hasattr(args, "network_type"):
        args.mode_task = args.network_type

    if not hasattr(args, "selection_threshold"):
        args.selection_threshold = None
        
    if not hasattr(args, "verbose"):
        args.verbose = 0
    if not hasattr(args, "bayesian"):
        args.bayesian = False

    if not hasattr(args, "prepare_dl"):
        if hasattr(args, "use_extracted_features"):
            args.prepare_dl = args.use_extracted_features
        elif hasattr(args, "use_extracted_patches") and args.mode == "patch":
            args.prepare_dl = args.use_extracted_patches
        elif hasattr(args, "use_extracted_slices") and args.mode == "slice":
            args.prepare_dl = args.use_extracted_slices
        elif hasattr(args, "use_extracted_roi") and args.mode == "roi":
            args.prepare_dl = args.use_extracted_roi

    return args



def show_fpg(data_batch, indices=None,plane="sag", num_rows=2, 
    num_cols=2, name=None, folder="/current_augmentations_examples/"):
    
    import matplotlib.pyplot as plt
    import numpy as np
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=((int(8 * num_rows), int(6 * num_cols))))
    data_batch=data_batch.numpy()
    print(data_batch.shape)
    data_batch=data_batch[:num_rows*num_cols].reshape(num_rows, num_cols, data_batch.shape[1],  data_batch.shape[2],  data_batch.shape[3], 
                                  data_batch.shape[4])
    print(data_batch.shape)

    for row in range(num_rows):
        for col in range(num_cols):

            i, j, k = indices
            data=data_batch[row][col]
            kwargs = dict(cmap='gray', interpolation='none')
            slices=dict()
            slices["sag"], slices["cor"], slices["axi"] = np.rot90(data[0, i]), np.rot90(data[0, :, j]), np.rot90(data[0, ..., k])

            axes[row][col].imshow(slices[plane],**kwargs)
            axes[row][col].axis('off')
            
#     path = '../../outputs/'+folder
#     if not os.path.exists(path):
#         os.makedirs(path)
    if name is not None:
        fig.suptitle(name)
    plt.subplots_adjust( left=0.05, right=0.95, top=0.95, bottom=0.05, wspace=0.05, hspace=0.05)

#     plt.savefig(path + str(name) + '.png')
    plt.show()
    plt.close()
    
def show_data(model_folder, name=None, plane="sag"):
    from tools.deep_learning.models import init_model
    from tools.deep_learning.data import (get_transforms,
                                        load_data,
                                        return_dataset,
                                        generate_sampler)
    from tools.deep_learning.iotools import return_logger
    from argparse import Namespace
    from torch.utils.data import DataLoader

    import torchvision.models
    import hiddenlayer as hl
    import torch

    path_params = os.path.join(model_folder, "commandline_train.json")
    with open(path_params, "r") as f:
        params = json.load(f)
    params = translate_parameters(Namespace(**params))
    main_logger = return_logger(params.verbose, "main process")

    
    train_transforms, all_transforms = get_transforms(params.mode,
                                                      minmaxnormalization=params.minmaxnormalization,
                                                      data_augmentation=None,
                                                      output_dir=None)
    training_df, valid_df = load_data(
            params.tsv_path,
            params.diagnoses,
            0,
            n_splits=params.n_splits,
            baseline=params.baseline,
            logger=main_logger
        )

    
    data_valid = return_dataset(params.mode, params.input_dir, valid_df, params.preprocessing,
                                train_transformations=train_transforms, all_transformations=all_transforms,
                                params=params)

    
    valid_loader = DataLoader(
        data_valid,
        batch_size=params.batch_size,
        shuffle=False,
        num_workers=params.num_workers,
        pin_memory=True
    )
    
    sample = next(iter(valid_loader))
    
    show_fpg(sample["image"], indices=(169//2, 208//2, 179//2), name=name, plane=plane)
    
    

In [ ]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ['1.5T', "3T","1.5T-3T" ]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:1]:
    print("MS %s \n ____________________________________________________________________________________________"%MS)
    model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3" ]
    MS_list = MS_list_dict[MS]
    
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments/Experiments-" + MS, "NNs" if isBayesian else "NNs")

    for network in model_types[:]:
        model_dir = os.path.join(model_dir_general, network)
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:1]:
            print(f)
            show_model(f)
#             show_data(f, plane="sag")
#             show_data(f, plane="cor")
#             show_data(f, plane="axi")
             

In [ ]:
hl_graph